In [1]:
#inititalize the notebook
import os
import textwrap
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

chat_deployment_name = "gpt-4"

print("*** init done! ***")


*** init done! ***


simple prompt

In [2]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Een HBO-ICT opleiding in Heerlen duurt normaal gesproken 4 jaar.


system prompt

In [3]:
question = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

response = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": question
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent die antwoord als een grappige kat"
        }
    ]
)

lines = textwrap.wrap(response.choices[0].message.content, width=80)
for line in lines:
    print(line)

Een HBO-ICT opleiding in Heerlen duurt doorgaans 4 jaar. Dit is als je
knetterhard studeert zonder ingeklemd te zijn tussen twee kussens in slaap
gevallen. Dus bereid je voor op veel muisklikken, toetsenbordtikken en misschien
wel wat nachtelijke muizenjachten!


RESET THE SYSTEM PROMPT

In [38]:
firstQuestion = "Hoe lang duurt een HBO-ICT opleiding in Heerlen?"

firstResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": firstQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(firstResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

In Nederland duurt een HBO-ICT opleiding (Hoger beroepsonderwijs - Informatie en
Communicatie Technologie) over het algemeen vier jaar. Dit geldt ook voor de
HBO-ICT opleiding in Heerlen, die bijvoorbeeld aangeboden kan worden door Zuyd
Hogeschool, een van de bekende aanbieders van hoger beroepsonderwijs in die
regio.  De vierjarige opleiding leidt op tot het niveau van bachelor en
combineert theoretische kennis met praktische vaardigheden op het gebied van IT.
Studenten kunnen meestal kiezen tussen verschillende specialisaties of
profielen, zoals software engineering, cyber security, data science, en
netwerktechnologie.  Het is ook mogelijk om een versneld traject te volgen als
je al relevante vooropleidingen of werkervaring hebt, waardoor je de opleiding
in sommige gevallen sneller kunt afronden. Daarnaast bieden sommige hogescholen
de mogelijkheid tot een deeltijdopleiding, wat langer kan duren, afhankelijk van
hoe de studie met werk en/of persoonlijk leven wordt gecombineerd.  Let 

vervolgvraag

In [39]:
secondQuestion = "Wat is het adres van de campus?"

secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Om je vraag correct te beantwoorden, moet ik weten naar welke campus je
specifiek verwijst, aangezien er wereldwijd veel universiteiten, hogescholen en
andere onderwijsinstellingen zijn, elk met hun eigen campussen.  Als je een
bepaalde instelling in gedachten hebt, zoals een universiteit of bedrijfscampus,
laat me dan de naam weten. Dan kan ik je het betreffende adres geven. Zonder
specifieke informatie kan ik helaas het adres niet verstrekken.


Add more context

In [40]:
systemWithMemory = {
    "role": "system",
    "content": f"""
            use the previous question and answer as a context 
            ##
            previous question {firstQuestion}
            previous answer {firstResponse.choices[0].message.content}
            """
}


secondResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    messages=[
        {
            "role": "user",
            "content": secondQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        },
        systemWithMemory
    ]
)

lines = textwrap.wrap(secondResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

Aangezien je refereert naar "de campus" zonder specifieke context, kan ik alleen
aannemen dat je het adres zoekt van de campus waar de HBO-ICT opleiding in
Heerlen wordt aangeboden. Als we ervan uitgaan dat de opleiding wordt aangeboden
door de Zuyd Hogeschool, dan is het adres van de campus in Heerlen:  Zuyd
Hogeschool Nieuw Eyckholt 300 6419 DJ Heerlen Nederland  Dit is het algemene
adres van de hoofdlocatie van Zuyd Hogeschool in Heerlen. Let wel op dat
onderwijsinstellingen soms meerdere locaties of faculteiten hebben, dus het is
raadzaam om de website van Zuyd Hogeschool te raadplegen of direct contact op te
nemen voor het meest specifieke en actuele adres gerelateerd aan de opleiding
die je zoekt.


short term memory - langchain

In [41]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()


# LLM
llm = AzureChatOpenAI(
    deployment_name=chat_deployment_name,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure",
) 


# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Je bent een behulpzame assistent"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
print("---------")
print(firstQuestion)
print("---------")
response = conversation({"question": firstQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)


print("---------")
print(secondQuestion + " (nu hebben we het antwoord van de eerste vraag als context)")
print("---------")
response = conversation({"question": secondQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)




---------
Hoe lang duurt een HBO-ICT opleiding in Heerlen?
---------
De HBO-ICT opleiding in Nederland duurt doorgaans vier jaar, wat overeenkomt met
de standaardduur voor voltijd bacheloropleidingen aan hogescholen. De Zuyd
Hogeschool is een van de instellingen in Heerlen die een HBO-ICT opleiding
aanbiedt. Als je bij Zuyd Hogeschool wilt studeren, kun je verwachten dat de
voltijd bacheloropleiding vier jaar duurt.  Er zijn echter verschillende
studiemodi beschikbaar, zoals deeltijd of duaal onderwijs, waarbij de opleiding
langer kan duren omdat je deze combineert met werk. Ook kunnen er mogelijkheden
zijn om de opleiding in een verkort traject te volgen, bijvoorbeeld als je al
relevante werkervaring hebt of een verwante vooropleiding hebt gevolgd.  Voor de
meest actuele informatie en eventuele specifieke programma's die worden
aangeboden, is het het beste om direct contact op te nemen met Zuyd Hogeschool
of de specifieke faculteit die de HBO-ICT opleiding verzorgt.
---------
Wat is h

Je kunt nu gewoon verder chatten, memory wordt steeds verder uitgebreid

Let op! Je prompt wordt steeds groter, dus je verbruikt meer tokens
Wellicht heb je niet alle historie van de chat nodig.

dit is ook een voorbeeld hoe je een specifieke instructie geeft. "antwoord alleen met de link naar Google Maps" is een hele specifieke instructie.

In [42]:
thirdQuestion = """
                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps
                """
print("---------")
print(thirdQuestion)
print("---------")
response = conversation({"question": thirdQuestion})

lines = textwrap.wrap(response["text"], width=80)
for line in lines:
    print(line)



---------

                    hoe zou een link naar Google Maps eruit zien voor dit adres?
                    antwoord alleen met de link naar Google Maps
                
---------
https://www.google.com/maps?q=Nieuw+Eyckholt+300,+6419+DJ+Heerlen,+Nederland


nog een voorbeeld van een hele specifieke instructie

In [43]:
JsonQuestion = """
                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                """
print("---------")
print(JsonQuestion)
print("---------")
response = conversation({"question": JsonQuestion})

print(response["text"])

---------

                    format your output in a json format
                    output a plain json as provided in the example with the address
                    ## sample json
                    {
                        "street": "street",
                        "city": "city",
                        "country": "country",
                        "postal_code": "postal_code"
                    }
                                        
                
---------
```json
{
    "street": "Nieuw Eyckholt 300",
    "city": "Heerlen",
    "country": "Nederland",
    "postal_code": "6419 DJ"
}
```


Een hele specifieke vraag

In [44]:
mySpecificQuestion = "wanneer is de open dag?"

mySpecificResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": mySpecificQuestion
        },
        {
            "role": "system",
            "content" : "Je bent een behulpzame assistent"
        }
    ]
)


lines = textwrap.wrap(mySpecificResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)



Om je vraag over de open dag te beantwoorden, heb ik meer context nodig. Open
dagen worden georganiseerd door verschillende instellingen zoals universiteiten,
scholen, bedrijven en organisaties. Elke instelling heeft zijn eigen schema voor
open dagen. Als je op zoek bent naar informatie over een specifieke open dag,
laat me dan weten om welke instelling of evenement het gaat, zodat ik je beter
kan helpen.


Wellicht kun je wat meer context geven in de prompt.

We gaan nu data toevoegen, hele specifieke data, namelijk een aantal pagina's
van de HSZuyd Website.

In [45]:
#lees de pagina's om deze later toe te voegen in de prompt
page1 = open("../docs/page-0.md", "r").read()
page2 = open("../docs/page-1.md", "r").read()
page3 = open("../docs/page-2.md", "r").read()

pages = [
    {
        "fileName": "page-0.md",
        "content": page1
    },
    {
        "fileName": "page-1.md",
        "content": page2
    },
    {
        "fileName": "page-2.md",
        "content": page3
    }
]
    

myDataQuestion = "wanneer is de open dag?"
try:
    myDataResponse = openai_client.chat.completions.create(
        model = chat_deployment_name,
        temperature=0.0,
        messages=[
            {
                "role": "user",
                "content": myDataQuestion
            },
            {
                "role": "system",
                "content" : f"""
                    Je bent een behulpzame assistent
                    Je geeft alleen antwoord op basis van de data die je hebt gekregen
                    Indien je het antwoord niet kunt vinden, zeg je dat je het antwoord niet weet
                    vermeld je bron document bij je antwoord

                    ## data
                    {pages}

                    """
            }
        ]
    )

    lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
    for line in lines:
        print(line)

    print("----")
    print("Hier wat info over de tokens die je hebt gebruikt:")
    print(myDataResponse.usage)
except Exception as e:
    print(e)

De open dag is op zaterdag 27 januari, van 10.00 tot 15.00 uur in Heerlen,
Maastricht en Sittard. Dit staat vermeld in de content van 'page-0.md'.
----
Hier wat info over de tokens die je hebt gebruikt:
CompletionUsage(completion_tokens=53, prompt_tokens=12788, total_tokens=12841)


Okay....dit lijkt werken, maar we verbruiken heel veel tokens...tijd voor een volgende stap.

***Embeddings en vectoren